In [ ]:
def garch1(prices,periods):
    results = holder()
    
    resdf = pd.DataFrame(index=prices.index)
    resdf['log_price'] = np.log(prices.close)
    resdf['pct_change'] = resdf['log_price'].pct_change()
    resdf['stdev21'] = resdf['pct_change'].rolling(21).std()
    resdf['hvol21'] = resdf['stdev21']*(252**0.5) # Annualize.
    resdf['variance'] = resdf['hvol21']**2
    resdf = resdf.dropna()
    
    returns = resdf['pct_change'] * 100
    am = arch.arch_model(returns)
    res = am.fit(disp='off')
    res.summary()

    resdf['forecast_var'] = res.conditional_volatility*(252**0.5)
    resdf['forecast_vol'] = 0.01 * np.sqrt(res.params['omega'] + res.params['alpha[1]'] * res.resid**2 + res.conditional_volatility**2*res.params['beta[1]'])

    results.df = resdf
    return results


def garch2(prices,periods):
    results = holder()
    
    w = pd.DataFrame(columns = ['date','C','H','L','R'])
    for j in range(periods[i]-1,len(prices)):
        C = prices.close.iloc[j]
        H = prices.high.iloc[j-periods[i]+1:j+1].max()
        am = arch.arch_model(prices.close, vol='Garch', p=1, o=0, q=1, dist='Normal')
    res = am.fit(update_freq=5,first_obs=j-periods[i],last_obs = j)
    forecasts = res.forecast()
    
    print(forecasts.mean.iloc[-3:])
    print(forecasts.residual_variance.iloc[-3:])
    print(forecasts.variance.iloc[-3:])
    
    
    return results    

def garch3(prices,periods):
    results = holder()
    
    prices = prices[-100:]
    resdf = pd.DataFrame(index=prices.index)
    resdf['close'] = prices.close
#    resdf['log_close'] = np.log(prices.close)*100
    resdf['returns'] =np.log(resdf.close/resdf.close.shift()) * 100
#    resdf['returns'] =(resdf.close/resdf.close.shift())
    plt.plot(resdf['returns'])
    #resdf = resdf.dropna()
    print(resdf)
    returns = resdf.returns
    am = arch.arch_model(returns[1:], vol='Garch', p=1, o=0, q=1, dist='Normal')

    split_date = dt.datetime(2018,10,1) # last observation really is 2018-09-31, first forcast is done for last observation
    split_date_idx = prices.index.get_loc(split_date)
#    res = am.fit(disp='off',last_obs=split_date)
    #print(res.summary())
#    forecasts = res.forecast( )

#    day = dt.timedelta(10)
#    print(resdf[split_date-day:])    

    resdf['h1'] = 0
    for i in range(0,len(periods)):
        for j in range(periods[i] - 1 + 1,len(prices)): 
            res = am.fit(disp='off',first_obs = j - periods[i] +1 ,last_obs = j)
            forecasts = res.forecast( )
            #print (forecasts.variance['h.1'])
            h1 = forecasts.variance['h.1']
            resdf.h1.iloc[j]=h1[j-1]
            

#    resdf['h1'] = forecasts.variance['h.1']
#    resdf['h1cumsum_shift'] = resdf.h1.cumsum().shift(1)
#    predictions_log = resdf.log_close.iloc[split_date_idx-1]
#    resdf['predictions_log'] = resdf.h1cumsum_shift.add(predictions_log,fill_value=0)
#    resdf['predictions']= np.exp(resdf['predictions_log'])
#    plt.plot(prices.close)
#    plt.plot(resdf['predictions'])
    resdf['h11'] = resdf.h1.shift(-1)
    print(resdf)
    plt.plot(resdf.h1)

    
    results.df = resdf
    return results

def arima1(prices,periods):
    '''
    działa na pliku
    prices = loaddata('USDJPY1440_airpass.csv')
    '''
    
    results = holder()
    
    resdf = pd.DataFrame(index=prices.index)
    resdf['close'] = prices.close
    '''
    albo log_close(wtedyd=1) albo returns(d=0 i dropna)
    '''
    resdf['log_close'] = np.log(prices.close)
    resdf['returns'] = resdf.log_close - resdf.log_close.shift()
#    resdf['returns'] = np.log(resdf.close/resdf.close.shift()) # to jest to samo co różnica logarytmów
    resdf.dropna(inplace=True)

    am = ARIMA(resdf.returns, order=(2, 0, 2))
    #am = ARIMA(resdf.log_close, order=(2, 1, 2))

    res = am.fit(disp=-1)
    forecasts = res.fittedvalues
    resdf['h1'] = forecasts
            

    resdf['h1cumsum_cumsum'] = resdf.h1.cumsum()
    predictions_log_base = resdf.log_close.iloc[0]
    resdf['predictions_log'] = resdf.h1cumsum_cumsum.add(predictions_log_base,fill_value=0)
    resdf['predictions'] = np.exp(resdf['predictions_log'])
    resdf['predictions'] = resdf.predictions
    resdf['predictions_shift'] = resdf.predictions.shift(1) # predykcja dotyczy następnej linijki

    plt.plot(resdf.close)
    plt.plot(resdf.predictions_shift)
    plt.show

    
    results.df = resdf
    return results


def arima(prices,periods):
    '''
    działa na pliku
    prices = loaddata('USDJPY1440_airpass.csv')
    wersja rolling
    '''
    
    results = holder()
    
    plot = True
    resdf = pd.DataFrame(index=prices.index)
    resdf['close'] = prices.close
    '''
    albo log_close(wtedy d=1) albo diff(d=0 i dropna)
    '''
    resdf['log_close'] = np.log(prices.close)
    resdf['returns'] = (resdf.log_close - resdf.log_close.shift())*1
#    resdf['returns'] = np.log(resdf.close/resdf.close.shift()) # to jest to samo co różnica logarytmów
    resdf.dropna(inplace=True)
#    resdf = resdf.append([resdf],ignore_index=True)

    resdf['h1'] = 0
    h1 = 0
    for i in range(0,len(periods)):
        for j in range(periods[i] - 1 ,len(resdf)): 
            first_obs = j - periods[i] + 1
            last_obs = j
            X = resdf.returns.values
            TS = X[first_obs:last_obs+1]                  
            h1 = predictARIMAOnSlice(TS,[2,0,2])
            resdf.h1.iloc[j]=h1

    resdf['h1cumsum'] = resdf.h1.cumsum()
    predictions_log_base = resdf.log_close.iloc[periods[i] - 1]
    resdf['predictions_log'] = resdf.h1cumsum.add(predictions_log_base,fill_value=0)
    resdf['predictions'] = np.exp(resdf['predictions_log'])
    resdf['predictions_shift'] = resdf.predictions.shift(1) # predykcja dotyczy następnej linijki

    if plot == True:
        plt.plot(resdf.close)
        plt.plot(resdf.predictions_shift)
        plt.show
    
    results.df = resdf
    return results

def predictARIMAOnSlice(TS,order):
    p_ = order[0]
    o_ = order[1]
    q_ = order[2]
    warnings.filterwarnings('ignore',category=Warning)
    with warnings.catch_warnings():
        warnings.simplefilter('error', OptimizeWarning)

        try:
            am = ARIMA(TS, order=(p_, o_, q_))
            res = am.fit(disp=0)
            forecasts = res.forecast()
            h1 = forecasts[0][0]
        except:
            h1 = np.NaN
    
    if math.isnan(h1):
        h1 = 0
        #h1 = last_h1
    
    return h1


def garch(prices,periods):
    '''
    działa na pliku
    prices = loaddata('USDJPY1440_airpass.csv')
    wersja rolling
    '''
    
    results = holder()
    
    plot = True
    resdf = pd.DataFrame(index=prices.index)
    resdf['close'] = prices.close
    '''
    albo log_close(wtedy d=1) albo diff(d=0 i dropna)
    '''
    resdf['log_close'] = np.log(prices.close)
    resdf['returns'] = (resdf.log_close - resdf.log_close.shift())*1
#    resdf['returns'] = np.log(resdf.close/resdf.close.shift()) # to jest to samo co różnica logarytmów
    resdf.dropna(inplace=True)
    resdf = resdf.append([resdf],ignore_index=True)

    resdf['h1'] = 0
    for i in range(0,len(periods)):
        for j in range(periods[i] - 1 ,len(resdf)): 
            first_obs = j - periods[i] + 1
            last_obs = j
            X = resdf.returns.values
            TS = X[first_obs:last_obs+1]        
            h1 = predictGARCHOnSlice(TS,[1,0,1])
            resdf.h1.iloc[j]=h1

    resdf['h1cumsum'] = resdf.h1.cumsum()
    predictions_log_base = resdf.log_close.iloc[periods[i] - 1]
    resdf['predictions_log'] = resdf.h1cumsum.add(predictions_log_base,fill_value=0)
    resdf['predictions'] = np.exp(resdf['predictions_log'])
    resdf['predictions_shift'] = resdf.predictions.shift(1) # predykcja dotyczy następnej linijki

    if plot == True:
        plt.plot(resdf.returns)
        plt.plot(resdf.h1)
        plt.show
    
    results.df = resdf
    return results
